In [1]:
from pathlib import Path
import numpy as np
# import h5py
from tqdm.auto import tqdm, trange
import scipy.io
import time
import torch
import random
import math
import shutil
from sklearn.decomposition import PCA
import scipy as sp
import scipy.signal
import os
import pandas as pd
from torch import nn
from scipy.spatial.distance import cdist
# import spikeinterface.core as sc
# import spikeinterface.full as si

from analysis.projections import pca_train, pca, get_flattened_data
from analysis.plotting import plot_gmm, plot_closest_spikes
from analysis.encoder_utils import load_GPT_backbone, get_fcenc_backbone
from analysis.cluster import GMM, HDBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score
from analysis.cluster import MeanShift
from analysis.benchmarking import class_scores, avg_score, per_class_accs, avg_class_accs
import matplotlib.patheffects as pe

from ceed.models.model_simclr import FullyConnectedEnc
from utils.utils import gmm_monitor, knn_monitor
from data_aug.wf_data_augs import Crop
from data_aug.contrastive_learning_dataset import WFDataset_lab 

/Users/ankit/miniforge3/envs/ceed/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ankit/miniforge3/envs/ceed/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ankit/miniforge3/envs/ceed/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be 

In [2]:
# class Args(Object):
#     def __init__(epoch_nums=1):
#         epochs = epoch_nums
    
class Args(object):
    pass

args = Args()
args.epochs = 1
args.use_chan_pos = False
args.use_gpt = False
print(args.epochs)

1


In [3]:
ten_neur_ood_path_dy016 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_10_neuron_400ood'
ten_neur_ood_path_dy009 = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy009_10_neuron_400ood'
fourhund_neur_ood_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real400n_200s'
sixhund_neur_ood_path = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/real600n_1200s'
test_setA = '/Users/ankit/Documents/PaninskiLab/contrastive_spikes/dy016_og_goodunit_subset'
test_fn = 'spikes_test.npy' 
train_fn = 'spikes_train.npy'

In [10]:
# dy016 ood data loaders
dy016_ood_5c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy016_ood_5c_memory_data, dy016_ood_5c_memory_labels = get_flattened_data(dy016_ood_5c_memory_dataset)
dy016_ood_5c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy016_ood_5c_test_data, dy016_ood_5c_test_labels = get_flattened_data(dy016_ood_5c_test_dataset)

dy016_ood_11c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy016_ood_11c_memory_data, dy016_ood_11c_memory_labels = get_flattened_data(dy016_ood_11c_memory_dataset)
dy016_ood_11c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy016, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy016_ood_11c_test_data, dy016_ood_11c_test_labels = get_flattened_data(dy016_ood_11c_test_dataset)

# dy009 ood data loaders
dy009_ood_5c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy009_ood_5c_memory_data, dy009_ood_5c_memory_labels = get_flattened_data(dy009_ood_5c_memory_dataset)
dy009_ood_5c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
dy009_ood_5c_test_data, dy009_ood_5c_test_labels = get_flattened_data(dy016_ood_5c_test_dataset)

dy009_ood_11c_memory_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy009_ood_11c_memory_data, dy009_ood_11c_memory_labels = get_flattened_data(dy009_ood_11c_memory_dataset)
dy009_ood_11c_test_dataset = WFDataset_lab(ten_neur_ood_path_dy009, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
dy009_ood_11c_test_data, dy009_ood_11c_test_labels = get_flattened_data(dy009_ood_11c_test_dataset)

# ten neuron ID data loaders
# dy009 ood data loaders
id_5c_memory_dataset = WFDataset_lab(test_setA, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
id_5c_memory_data, id_5c_memory_labels = get_flattened_data(id_5c_memory_dataset)
id_5c_test_dataset = WFDataset_lab(test_setA, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=2, ignore_chan_num=True), use_chan_pos=False)
id_5c_test_data, id_5c_test_labels = get_flattened_data(id_5c_test_dataset)

id_11c_memory_dataset = WFDataset_lab(test_setA, split='train', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
id_11c_memory_data, id_11c_memory_labels = get_flattened_data(id_11c_memory_dataset)
id_11c_test_dataset = WFDataset_lab(test_setA, split='test', multi_chan=True, transform=Crop(prob=0.0, num_extra_chans=5, ignore_chan_num=True), use_chan_pos=False)
id_11c_test_data, id_11c_test_labels = get_flattened_data(id_11c_test_dataset)

True spikes_train.npy
(2000, 21, 121)
(2000, 21, 121)
True spikes_train.npy
(2000, 21, 121)
(2000, 21, 121)
True spikes_train.npy
(2000, 21, 121)
(2000, 21, 121)
True spikes_train.npy
(2000, 21, 121)
(2000, 21, 121)
True spikes_train.npy
(2000, 21, 121)
(2000, 21, 121)
True spikes_train.npy
(2000, 21, 121)
(2000, 21, 121)


In [ ]:
def pca_gmm_average(memory_dataset, test_dataset, test_labels, num_classes=10, num_iters=50, out_dim=5):
    scores = []
    for i in range(num_iters):
        # covariance_type : {'full', 'tied', 'diag', 'spherical'}
        covariance_type = 'full'
        reps_train, _, pca_ = pca(memory_dataset, out_dim)
        reps_test = pca_.transform(test_dataset)
        gmm = GaussianMixture(num_classes,
                            random_state=random.randint(0, 1000000),
                            covariance_type=covariance_type).fit(reps_train)
        gmm_cont_test_labels = gmm.predict(reps_test)
        curr_score = adjusted_rand_score(test_labels, gmm_cont_test_labels)*100
        scores.append(curr_score)
        if i == 49:
           print("max gmm score: {}".format(max(scores)))
           print("min gmm score: {}".format(min(scores)))
           print("50 run gmm mean score: {}".format(np.mean(scores)))
           print("50 run gmm std-dev score: {}".format(np.std(scores)))

    score = np.mean(scores)

    return score

In [11]:
dy016_ood_score = pca_gmm_average(dy016_ood_5c_memory_data, dy016_ood_5c_test_data, dy016_ood_5c_test_labels)
print("DY016 OOD data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_ood_5c_memory_data, dy009_ood_5c_memory_data, dy009_ood_5c_test_labels)
print("DY009 OOD data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_5c_memory_data, id_5c_test_data, id_5c_test_labels)
print("test set A data GMM score: " + str(testset_A_score))

KeyboardInterrupt: 

In [ ]:
dy016_ood_score = pca_gmm_average(dy016_ood_11c_memory_data, dy016_ood_11c_test_data, dy016_ood_11c_test_labels)
print("DY016 OOD data GMM score: " + str(dy016_ood_score))
dy009_ood_score = pca_gmm_average(dy009_ood_11c_memory_data, dy009_ood_11c_memory_data, dy009_ood_11c_test_labels)
print("DY009 OOD data GMM score: " + str(dy009_ood_score))
testset_A_score = pca_gmm_average(id_11c_memory_data, id_11c_test_data, id_11c_test_labels)
print("test set A data GMM score: " + str(testset_A_score))